<a href="https://colab.research.google.com/github/Anilesh05/Big_Data_Laboratory/blob/main/Find_grade_Using_Mapreduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Downloading and installing hadoop***

In [13]:
!apt-get install openjdk-8-jdk
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar fx hadoop-3.3.6.tar.gz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
!ln -s /content/hadoop-3.3.6/bin/* /usr/bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-8-jdk is already the newest version (8u402-ga-2ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
--2024-04-03 14:27:55--  https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz.1’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  45.3MB/s    in 6.6s    

2024-04-03 14:28:01 (106 MB/s) - ‘hadoop-3.3.6.tar.gz.1’ saved [730107476/730107476]

ln: failed to create symbolic link '/usr/bin/container-executor': File exists
ln: failed to create symbolic link '/usr/bin/hadoop': File exists
ln: failed to create symbolic link '/usr/bin/hadoop.cmd': File exists

# ***Create Input Directory***

In [ ]:
!hdfs dfs -mkdir input

# ***Create mapper.py***

In [34]:
%%writefile mapper.py
#!/usr/bin/env python

import sys

# Define column names
column_names = ["Student ID", "Name", "Total Marks", "Grade"]

# Skip the header row
header_skipped = False

# Input comes from STDIN (standard input)
for line in sys.stdin:
    # Remove leading and trailing whitespace
    line = line.strip()

    if not header_skipped:
        # Skip the header row
        header_skipped = True
        continue

    # Split the line into columns
    columns = line.split(',')
    if len(columns) == 3:
        student_id, name, marks = columns
        total_marks = sum(map(float, [marks]))
        # Determine the grade
        if total_marks >= 90:
            grade = "A+"
        elif total_marks >= 80:
            grade = "A"
        elif total_marks >= 70:
            grade = "B+"
        elif total_marks >= 60:
            grade = "B"
        elif total_marks >= 50:
            grade = "C"
        else:
            grade = "U"
        # Output the result
        print(f"{student_id},{name},{total_marks},{grade}")


Overwriting mapper.py


# ***Create Reducer.py***

In [29]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

# Input comes from STDIN (standard input)
for line in sys.stdin:
    # Remove leading and trailing whitespace
    line = line.strip()
    # Split the line into columns
    student_id, name, total_marks, grade = line.split(',')
    # Output in table format
    print(f"{student_id:<5} {name:<15} {total_marks:<10} {grade}")


Overwriting reducer.py


# ***Write input file***

In [39]:
%%writefile input/marksheet.txt

01,John,85
02,Emma,92
03,Michael,78
04,Sophia,60
05,William,88
06,Isabella,75
07,James,90
08,Olivia,82
09,Alexander,70
10,Ava,95
11,Ethan,72
12,Mia,86
13,Daniel,81
14,Charlotte,79
15,Joseph,94
16,Abigail,87
17,David,76
18,Emily,93
19,Benjamin,83
20,Harper,77
21,Matthew,91
22,Elizabeth,84
23,Jackson,34
24,Ella,89
25,Samuel,80
26,Avery,96
27,Henry,73
28,Scarlett,97
29,Gabriel,68
30,Victoria,98
31,Andrew,40
32,Grace,99
33,Ryan,65
34,Chloe,100
35,Lucas,66
36,Sofia,98
37,Christopher,67
38,Liam,97
39,Evelyn,64
40,Dylan,96

Overwriting input/marksheet.txt


# ***Run hadoop mapreduce***

In [41]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -files mapper.py,reducer.py \
    -mapper mapper.py \
    -reducer reducer.py \
    -input input/marksheet.txt \
    -output output

2024-04-03 14:46:32,014 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-04-03 14:46:32,124 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-04-03 14:46:32,124 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-04-03 14:46:32,144 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-04-03 14:46:32,465 INFO mapred.FileInputFormat: Total input files to process : 1
2024-04-03 14:46:32,487 INFO mapreduce.JobSubmitter: number of splits:1
2024-04-03 14:46:32,685 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1277131040_0001
2024-04-03 14:46:32,686 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-04-03 14:46:33,135 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local1277131040_0001_f5b40699-43fc-4078-bf31-f8718b5b756f/mapper.py
2024-04-03 14:46:33,152 INFO mapred.LocalDistributedCacheMa

# ***Display Output***

In [42]:
!cat output/part-00000

01    John            85.0       A	
02    Emma            92.0       A+	
03    Michael         78.0       B+	
04    Sophia          60.0       B	
05    William         88.0       A	
06    Isabella        75.0       B+	
07    James           90.0       A+	
08    Olivia          82.0       A	
09    Alexander       70.0       B+	
10    Ava             95.0       A+	
11    Ethan           72.0       B+	
12    Mia             86.0       A	
13    Daniel          81.0       A	
14    Charlotte       79.0       B+	
15    Joseph          94.0       A+	
16    Abigail         87.0       A	
17    David           76.0       B+	
18    Emily           93.0       A+	
19    Benjamin        83.0       A	
20    Harper          77.0       B+	
21    Matthew         91.0       A+	
22    Elizabeth       84.0       A	
23    Jackson         34.0       U	
24    Ella            89.0       A	
25    Samuel          80.0       A	
26    Avery           96.0       A+	
27    Henry           73.0       B+	
28    Scarlet